In [1]:
%load_ext autoreload
%autoreload 2
import config as cfg
from sca import plots, helpers as h

In [2]:
X, y, pts, ks = h.load_data(cfg.DATA_DIR / "ascadv_clean.h5")
fi, feat_rank = h.rf_feature_importances(X, y, pts, ks, seed=cfg.SEED)

2026-01-21 01:43:06: [1] Mean PI: 0.002


In [3]:
prof_idx, atk_idx = h.split_idx(X, y, seed=cfg.SEED)
X_prof, y_prof = X[prof_idx], y[prof_idx]
X_atk, y_atk, pts_atk, ks_atk = X[atk_idx], y[atk_idx], pts[atk_idx], ks[atk_idx]

In [4]:
import lightgbm as lgb
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def PI_lgb(pts_val, ks_val):
    def fn(y_true, y_pred):
        y_pred_reshaped = y_pred.reshape((-1, 256), order='F')
        score = np.mean(h.PI(y_pred_reshaped, pts_val, ks_val))
        return ('PI', score, False)
    return fn

lgbc = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='multiclass',
    data_sample_strategy='goss',
    max_bin=63,   
    n_jobs=-1, 
    colsample_bytree=0.8, 
    num_leaves=7,
    n_estimators=1000,
    learning_rate=0.05,
    min_data_in_leaf=100,
    reg_alpha=1.0,
    verbose=1
)

In [6]:
lgbc.fit(
    X_prof, y_prof,
    callbacks=[h.LGBValidateEvery(
        X_atk, pts_atk, ks_atk,
        patience=50, heck_every=10)
    ]
)

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.334430 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24332
[LightGBM] [Info] Number of data points in the train set: 99999, number of used features: 1400
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Info] Start training from score -5.572744
[LightGBM] [Info] Start training from score -5.562273
[LightGBM] [Info] Start training from score -5.544208
[LightGBM] [Info] Start training from score -5.557078
[LightGBM] [Info] Start training from score -5.465543
[LightGBM] [Info] Start training from

NameError: name 'lgb' is not defined